In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import scipy.stats as sst

In [ ]:
def do_one_exp(n, mu, sigma, alpha, dist='normal'):
    """
    Parameters:
    -----------
    n: int,
        The number of sample in the experiment
    mu: float
        The mean of the alternative
    sigma: float
        The standard deviation of the alternative
    alpha: float
        The risk of error (type I)
        
    Returns:
    --------
    (2,) tuple : (effect, t_value)
    """
    
    try:
        assert dist in ["normal"] 
    except AssertionError as e:        
        raise e #(e.message + ' happens at %s' % dist)
  
    dist_rv = sst.norm(mu, sigma)
    sample = dist_rv.rvs(size=(n,))

    effect = sample.mean()

    std_error_data = np.std(sample, ddof=1)
    std_error_mean = std_error_data/np.sqrt(n)

    t_value = effect/std_error_mean

    return (effect, t_value)


In [ ]:
# a quick function to compute power
def stat_power(n, mu, sigma, alpha, dist='normal'):
    """
    Parameters:
    -----------
    n: int,
        The number of sample in the experiment
    mu: float
        The mean of the alternative
    sigma: float
        The standard deviation of the alternative
    alpha: float
        The risk of error (type I)
    
    Returns:
    --------
    float
        The statistical power for this number of sample, mu, sigma, alpha    """
    
    df = n-1
    # theta tells us how 'far' is the alternative hypothesis
    theta = np.sqrt(n)*mu/sigma
    
    # this is the alternative 
    ncrv = sst.nct(df, theta)

    # this is the threshold, computed on the null
    t_alph_null = sst.t.isf(alpha, df)

    spow = 1 - ncrv.cdf(t_alph_null)

    return spow


In [ ]:
args = [30,.25,1,.05]
print(do_one_exp(*args, dist='normal'))
print(stat_power(*args))

In [ ]:
# do N_exp experiments

N_exp = 100
effects = np.zeros((N_exp,))
t_values = np.zeros((N_exp,))

n = 30
mu = .25
alpha = 0.05
sigma = 1

for exp in range(N_exp):
    effects[exp], t_values[exp] = do_one_exp(n, mu, sigma, alpha)

strv = sst.t(n-1) #degree of freedom of the student distribution is n-1
t_alpha = strv.isf(alpha)

published = effects[t_values > t_alpha]

print(effects.mean(), published.mean())